<a href="https://colab.research.google.com/github/redperiabras-dezoomcamp/workshop-1/blob/main/Homework_data_talks_club_data_extraction_and_ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [31]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

In [32]:

# Answer 1

limit = 5
generator = square_root_generator(limit)
sum_sqrt = sum([i for i in generator])
print(sum_sqrt)

8.382332347441762


In [33]:
# Answer 2

limit = 13
generator = square_root_generator(limit)
sqrts = [i for i in generator]
print(sqrts[-1])

3.605551275463989


`

# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [34]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [35]:
import dlt

In [36]:
@dlt.resource(table_name='people', write_disposition='replace')
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

@dlt.resource(table_name='people', write_disposition='append')
def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


In [37]:
pipeline = dlt.pipeline(
    pipeline_name='people_pipeline',
    destination='duckdb',
    dataset_name='people_dataset',
    progress=dlt.progress.tqdm(colour="yellow")
)

In [38]:
info = pipeline.run(people_1)
info = pipeline.run(people_2)

Extract people: Resources:   0%|          | 0/1 [00:00<?, ?it/s]
people: 0it [00:00, ?it/s]
                                                                        
people: 5it [00:00, 588.79it/s]
Normalize people in 1707791196.2929938: Files:   0%|          | 0/1 [00:00<?, ?it/s]
Items: 0it [00:00, ?it/s]
Normalize people in 1707791196.2929938: Files: 100%|██████████| 1/1 [00:00<00:00,  2.91it/s]
                                                                                            
Items: 5it [00:00, 14.87it/s]
Extract people: Resources:   0%|          | 0/1 [00:00<?, ?it/s]
people: 0it [00:00, ?it/s]
                                                                        
people: 6it [00:00, 458.57it/s]
Normalize people in 1707791197.156925: Files:   0%|          | 0/1 [00:00<?, ?it/s]
Items: 0it [00:00, ?it/s]
Normalize people in 1707791197.156925: Files: 100%|██████████| 1/1 [00:00<00:00,  2.89it/s]
                                                                             

In [41]:
# Answer 3

import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

query = conn.sql("SELECT SUM(age) FROM people AS SUM_AGE").df()
display(query)

,sum(age)
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [42]:
import duckdb

pipeline = dlt.pipeline(destination='duckdb', dataset_name='people_dataset')

info = pipeline.run(people_1(),
                    table_name="people1",
					write_disposition="replace")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

# we can load the next generator to the same table.
info = pipeline.run(people_2(),
                    table_name="people1",
                    write_disposition="merge",
                    primary_key="id")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.40 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_dataset
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707791549.1174953 is LOADED and contains no failed jobs
Pipeline dlt_colab_kernel_launcher load step completed in 0.63 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_dataset
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707791549.9712248 is LOADED and contains no failed jobs


In [45]:
# Answer 4

import duckdb

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")

query = conn.sql("SELECT SUM(age) FROM people1 AS SUM_AGE").df()
display(query)

,sum(age)
0,266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX